# TSA Chapter 5: Volatility Clustering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_clustering/TSA_ch5_clustering.ipynb)

S&P 500 returns and GARCH(1,1) conditional volatility showing volatility clustering.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
import yfinance as yf

# Download S&P 500 data
sp = yf.download('^GSPC', start='2000-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
sp500_returns = (sp['Close'].pct_change() * 100).dropna()
sp500_returns = pd.Series(sp500_returns.values, index=sp500_returns.index, name='returns')
print(f"S&P 500: {len(sp500_returns)} observations")
from arch import arch_model

# Fit GARCH models on S&P 500
m_g = arch_model(sp500_returns, vol='Garch', p=1, q=1, dist='normal')
r_g = m_g.fit(disp='off')
m_j = arch_model(sp500_returns, vol='Garch', p=1, o=1, q=1, dist='normal')
r_j = m_j.fit(disp='off')
m_e = arch_model(sp500_returns, vol='EGARCH', p=1, q=1, dist='normal')
r_e = m_e.fit(disp='off')
models = {'garch': r_g, 'gjr': r_j, 'egarch': r_e}
print("GARCH models fitted successfully")

In [ ]:
# Volatility clustering plot
cond_vol = models['garch'].conditional_volatility

fig, axes = plt.subplots(2, 1, figsize=(12, 7), sharex=True)
axes[0].plot(sp500_returns.index, sp500_returns.values, color=BLUE, lw=0.4, alpha=0.8, label='Daily Returns')
axes[0].axhline(0, color='black', lw=0.5)
axes[0].set_ylabel('Daily Return (%)'); axes[0].set_title('S&P 500 - Daily Returns (Real Data)', fontweight='bold')

crises = [('2008-09-01', '2009-03-31', '2008 Crisis'), ('2020-02-20', '2020-04-30', 'COVID-19'), ('2022-01-01', '2022-10-31', '2022')]
for s, e, lab in crises:
    axes[0].axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.15, color=RED)
    axes[1].axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.15, color=RED)

axes[1].plot(sp500_returns.index, cond_vol, color=RED, lw=0.6, label='GARCH(1,1) volatility')
axes[1].fill_between(sp500_returns.index, 0, cond_vol, color=RED, alpha=0.2)
axes[1].set_ylabel('Conditional Volatility (%)'); axes[1].set_xlabel('Date')
axes[1].set_title('Volatility Clustering - GARCH(1,1)', fontweight='bold')

for ax in axes:
    ax.xaxis.set_major_locator(mdates.YearLocator(4))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

handles0, labels0 = axes[0].get_legend_handles_labels()
handles1, labels1 = axes[1].get_legend_handles_labels()
fig.legend(handles0 + handles1, labels0 + labels1,
           loc='upper center', bbox_to_anchor=(0.5, -0.02), ncol=3, frameon=False)
fig.tight_layout(); fig.subplots_adjust(bottom=0.10)
save_chart(fig, 'garch_volatility_clustering'); plt.show()